In [1]:
from scipy.stats import norm
likehood = norm.pdf(5.2476967371604015, 6, 3)
print('likehod: ',likehood)
prob = 0.0025 * likehood
print('prob: ', prob)


likehod:  0.12886459782042922
prob:  0.00032216149455107306


In [ ]:
prior = 1 /(gridSize * gridSize)

    for x in range(gridSize):
        for y in range(gridSize):
            prob = prior
            for t in range(observation.shape[0]):
                agentX = observation.iloc[t, 0]
                agentY = observation.iloc[t, 1]
                eDist  = observation.iloc[t, 2]

                dx = abs(agentX - x)
                dy = abs(agentY - y)
                dx = min(dx, (gridSize - dx))
                dy = min(dy, (gridSize - dy))

                distance = math.sqrt(dx**2 + dy**2)

                likehood = norm.pdf(eDist, distance, std)
                prob *= likehood
                p_map[x, y] = prob

    total = np.sum(p_map)
    if total > 0:
        p_map /= total

    print(p_map)

1.3333333333333333


In [ ]:
# Function: Get Belief
# ---------------------
# Updates beliefs based on the distance observation and your car's (agent's) position.
# Returns your belief of the probability that the car is in each tile. Your
# belief probabilities should sum to 1.
# @return the probability matrix
def getBelief(observation, gridSize, carLength):
    std = carLength * (1/3)

    #your code
    # p_map = np.zeros((gridSize, gridSize))

    
    # Initialize a uniform prior over all grid locations
    # p_map[i][j] represents the probability that the hidden car is at row i, column j.
    p_map = np.ones((gridSize, gridSize), dtype=float) / (gridSize * gridSize)

    # If the observation is provided as a DataFrame, iterate over its rows.
    # Each row gives the agent's position (agentX, agentY) and a noisy distance measurement (eDist).
    # For a stationary car, we update the belief by multiplying the prior by the likelihood
    # of observing each distance measurement given the hidden car location.
    for _, row in (observation.iterrows() if isinstance(observation, pd.DataFrame) else enumerate(observation)):
        agent_x = int(row['agentX'])
        agent_y = int(row['agentY'])
        measured_dist = row['eDist']

        # Update the belief for every possible car position (i, j)
        for i in range(gridSize):
            for j in range(gridSize):
                # Compute toroidal (periodic) distance in x and y directions
                dx = min(abs(agent_x - i), gridSize - abs(agent_x - i))
                dy = min(abs(agent_y - j), gridSize - abs(agent_y - j))
                true_dist = math.sqrt(dx * dx + dy * dy)
                # Likelihood of the observed distance given the true distance
                likelihood = norm.pdf(measured_dist, loc=true_dist, scale=std)
                # Multiply the prior belief by the likelihood
                p_map[i, j] *= likelihood

        # Normalize the probabilities to sum to 1 after each observation to avoid underflow
        total = p_map.sum()
        if total > 0:
            p_map /= total

    return p_map
